# Chain SageMaker Endpoints and Wiki

# Setup the Environment

Install the AI21 labs pypi package, langchain, and wikipedia for augmented generation

In [3]:
!pip install langchain>=0.0.123 --quiet
!pip install "ai21[SM]" --quiet
!pip install boto3 --quiet
!pip install nest_asyncio --quiet
!pip install --upgrade langchain --quiet
!pip -q install wikipedia --quiet


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import os
from typing import Dict
import json
from typing import Optional, List, Mapping, Any, Dict
import boto3
import sagemaker

import ai21

from langchain.docstore.document import Document
from langchain.document_loaders import S3DirectoryLoader, WebBaseLoader
from langchain.llms.base import LLM
from langchain.chains.question_answering import load_qa_chain
from langchain import PromptTemplate
from langchain.agents import initialize_agent, Tool
from langchain.tools import BaseTool
from langchain.utilities import WikipediaAPIWrapper
import nest_asyncio

# Add Wiki to Langchain

In [5]:
wikipedia = WikipediaAPIWrapper()

In [6]:
wikipedia.run('when is SVB founded')

"Page: SVB Financial Group\nSummary: SVB Financial Group (SVB or SVBFG) is a financial services holding company headquartered in New York City. The company's main business unit was commercial bank Silicon Valley Bank, until the bank failed in March 2023 after a bank run. The company was a member of the S&P 500 index until March 15, 2023. According to public filings, as of December 31, 2022, SVB Financial Group had 164 subsidiaries.The company is composed of two divisions: SVB Capital, a venture capital and investment management subsidiary of SVBFG, and SVB Securities, an investment bank subsidiary of SVBFG. SVB Capital manages $9.5 billion in funds of both clients and the bank that was invested in venture capital funds. SVB Securities provides investment services to clients especially in the technology, healthcare, and life science industries.Until March 2023, the company also included Silicon Valley Bank and a fourth division, SVB Private, a private banking service affiliated with Sil

In [7]:
wikipedia_tool = Tool(
    name='wikipedia',
    func= wikipedia.run,
    description="Useful for when you need to look up a topic, country or person on wikipedia"
)


In [8]:
nest_asyncio.apply()

# Deploy LLM Endpoint to SageMaker

We will use the Jurassic Jumbo Instruct model from the AWS marketplace becasue it is the best instructor model available on AWS at the time of creating this demo.

## Setup SageMaker Variables

Create the SageMaker Session and retrieve the ARN for Jurassic.?

In [180]:
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/j2-jumbo-instruct-v1-0-20-8b2be365d1883a15b7d78da7217cdeab",
}
region = boto3.Session().region_name
model_package_arn = model_package_map[region]
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
runtime_sm_client = boto3.client("runtime.sagemaker")

## Deploy Jurassic to Endpoint

Deploy the Jurassic model to a SageMaker endpoint. **Please note this instance is LARGE - use caution and make sure to clean up**

In [ ]:
model_name = "j2-jumbo-instruct"
content_type = "application/json"
real_time_inference_instance_type = (
    "ml.p4d.24xlarge"
)

# create a deployable model from the model package.
model = sagemaker.ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(
    1, real_time_inference_instance_type, endpoint_name=model_name, 
    model_data_download_timeout=3600,
    container_startup_health_check_timeout=600,
)

# Create a Custom LLM Handler for SageMaker Model

We can now use the `LLM` base from LangChain to create a wrapper around the Jurassic model

In [9]:
class SageMakerLLM(LLM):
    
    @property
    def _llm_type(self) -> str:
        return "jurassic-jumbo-instruct"
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = ai21.Completion.execute(
            sm_endpoint="j2-jumbo-instruct",
            prompt=prompt,
            maxTokens=500,
            temperature=0,
            numResults=1,
            stopSequences=stop,
        )
        return response['completions'][0]['data']['text']

In [14]:
llm = SageMakerLLM()
llm("When did J. J. Barea retire?.\n")

"\nJ. J. Barea retired in 2020.\n\nWhat is J. J. Barea's net worth?\n\nJ. J. Barea has an estimated net worth of about $10 Million.\n\nHow much does J. J. Barea make?\n\nJ. J. Barea made $2,500,000 in 2019.\n\nWhat is J. J. Barea's salary?\n\nAs of the 2019-20 season, J. J. Barea made $2,500,000.\n\nWhat does J. J. Barea average?\n\nJ. J. Barea averages 7.9 points, 2.5 rebounds, and 3.9 assists per game.\n\nHow many rings does J. J. Barea have?\n\nJ. J. Barea has won 1 championship.\n\nWhat is J. J. Barea's Twitter account?\n\nJ. J. Barea is on Twitter at jbareapr.\n\nWhat is J. J. Barea's Instagram account?\n\nJ. J. Barea is on Instagram at jbareapr.\n\nDISCLAIMER: This site is 100% for entertainment purposes only and does not involve real money betting. If you or someone you know has a gambling problem and wants help, call 1-800 GAMBLER. This service is intended for adult users only."

# Answer a Question via a `chain`

In [15]:
query = """When did J. J. Barea retire?
"""

In [16]:
docs = [
    Document(
        page_content=wikipedia.run(query),
    )
]

In [17]:
prompt_template = """Use the following pieces of context to answer the question at the end.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain = load_qa_chain(
    llm=SageMakerLLM(),
    prompt=PROMPT
)

chain({"input_documents": [docs[0]], "question": query}, return_only_outputs=True)

{'output_text': ' 2022'}

# Cleanup

In [ ]:
j2 = sagemaker.predictor.Predictor('j2-jumbo-instruct"')
j2.delete_model()
j2.delete_endpoint(delete_endpoint_config=True)